In [1]:
# Change directory to the root of the project
import os 
os.chdir('..')
os.chdir('..')
os.chdir('..')
print(f"Working directory: {os.getcwd()}")

Working directory: /Users/eohjelle/Documents/2025-dots-and-boxes/dots-and-boxes


In [2]:
from applications.dots_and_boxes.NNmodels.MLP import MLPInitParams
import torch

# Initialize parameters

## Model parameters
model_type = 'mlp'
model_params: MLPInitParams = {
    'num_rows': 3,
    'num_cols': 2,
    'hidden_sizes': [64, 128, 256, 128, 64]
}
device = torch.device('mps')
model_name = 'dots_and_boxes_mlp'

## Initialize new model
load_model = None
load_model_params = {}

## Optimizer parameters
optimizer_type = 'adam'
optimizer_params = {
    'lr': 1e-2,
    'betas': (0.9, 0.999),
    'eps': 1e-8,
    'weight_decay': 1e-4,
    'amsgrad': False
}

## Learning scheduler parameters
lr_scheduler_type = 'plateau'
lr_scheduler_params = {
    'factor': 0.5,
    'patience': 25,
    'cooldown': 50,
    'min_lr': 1e-6
}

## Training parameters
training_method = 'supervised'
trainer_params = {}
training_params = {
    'epochs': 50,
    'batch_size': 256,
    'eval_freq': 25,
    'checkpoint_freq': 50,
    'mask_illegal_moves': False,
    'mask_value': -20.0, # Doesn't matter when mask_illegal_moves is False
    'checkpoint_dir': 'checkpoints',
    'start_at': 1
}

## Load replay buffer from wandb
load_replay_buffer = 'from_file'
load_replay_buffer_params = {
    'path': f'applications/dots_and_boxes/training_data/dots_and_boxes_{model_params["num_rows"]}x{model_params["num_cols"]}_DABSimpleTensorMapping_minimax.pkl',
    'device': device
}
# load_replay_buffer_params = {
#     'project': 'AlphaZero-DotsAndBoxes',
#     'artifact_name': f'dots_and_boxes_{model_params["num_rows"]}x{model_params["num_cols"]}_SimpleTensorMapping_minimax',
#     'artifact_version': 'latest'
# }


In [ ]:
# Initialize wandb run
import wandb

run_name = 'MLP 4'
notes = 'MLP 2 for 3 x 2 board.'

config = {
    'model_type': model_type,
    'model_params': model_params,
    'optimizer_type': optimizer_type,
    'optimizer_params': optimizer_params,
    'lr_scheduler_type': lr_scheduler_type,
    'lr_scheduler_params': lr_scheduler_params,
    'training_method': training_method,
    'trainer_params': trainer_params,
    'training_params': training_params
}

run = wandb.init(
    project='AlphaZero-DotsAndBoxes',
    name=run_name,
    config=config,
    notes=notes
)
# run = None

wandb: Currently logged in as: eohjelle (eigenway) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [4]:
# Perform training

from applications.dots_and_boxes.train import train

model_interface = train(
    model_type=model_type,
    model_params=model_params,
    device=device,
    model_name=model_name,
    optimizer_type=optimizer_type,
    optimizer_params=optimizer_params,
    lr_scheduler_type=lr_scheduler_type,
    lr_scheduler_params=lr_scheduler_params,
    training_method=training_method,
    trainer_params=trainer_params,
    training_params=training_params,
    load_model=load_model,
    load_model_params=load_model_params,
    load_replay_buffer=load_replay_buffer,
    load_replay_buffer_params=load_replay_buffer_params,
    wandb_run=run
)

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/Users/eohjelle/Documents/2025-dots-and-boxes/dots-and-boxes/core/data_structures.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to

Epoch 1/50, Train Loss: 2.4671, Val Loss: 2.3989, Learning rate: 0.010000
Epoch 2/50, Train Loss: 2.3674, Val Loss: 2.3146, Learning rate: 0.010000
Epoch 3/50, Train Loss: 2.2602, Val Loss: 2.2207, Learning rate: 0.010000
Epoch 4/50, Train Loss: 2.1956, Val Loss: 2.1846, Learning rate: 0.010000
Epoch 5/50, Train Loss: 2.1581, Val Loss: 2.1450, Learning rate: 0.010000
Epoch 6/50, Train Loss: 2.1259, Val Loss: 2.1257, Learning rate: 0.010000
Epoch 7/50, Train Loss: 2.0911, Val Loss: 2.0703, Learning rate: 0.010000
Epoch 8/50, Train Loss: 2.0415, Val Loss: 2.0316, Learning rate: 0.010000
Epoch 9/50, Train Loss: 2.0099, Val Loss: 1.9912, Learning rate: 0.010000
Epoch 10/50, Train Loss: 1.9699, Val Loss: 1.9837, Learning rate: 0.010000
Epoch 11/50, Train Loss: 1.9611, Val Loss: 1.9520, Learning rate: 0.010000
Epoch 12/50, Train Loss: 1.9218, Val Loss: 1.9153, Learning rate: 0.010000
Epoch 13/50, Train Loss: 1.8945, Val Loss: 1.8931, Learning rate: 0.010000
Epoch 14/50, Train Loss: 1.8669, V

In [5]:
run.finish()

Minimax_draw_rate,▁█
Minimax_loss_rate,█▁
Minimax_score,▁█
Minimax_win_rate,▁█
RandomAgent_draw_rate,▁█
RandomAgent_loss_rate,█▁
RandomAgent_score,▁█
RandomAgent_win_rate,▁█
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▇▆▅▅▄▄▄▃▃▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [6]:
def print_model_parameters(model):
    """
    Print the total number of parameters in a PyTorch model,
    with a breakdown of trainable vs non-trainable parameters.
    
    Args:
        model: PyTorch model
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%})")
    print(f"Non-trainable parameters: {non_trainable_params:,} ({non_trainable_params/total_params:.2%})")
    
    # Optional: Print parameters by layer
    print("\nParameters by layer:")
    for name, param in model.named_parameters():
        print(f"{name}: {param.numel():,} parameters")

# Example usage
print_model_parameters(model_interface.model)

Total parameters: 84,173
Trainable parameters: 84,173 (100.00%)
Non-trainable parameters: 0 (0.00%)

Parameters by layer:
layers.0.fc1.weight: 768 parameters
layers.0.fc1.bias: 64 parameters
layers.1.fc1.weight: 8,192 parameters
layers.1.fc1.bias: 128 parameters
layers.2.fc1.weight: 32,768 parameters
layers.2.fc1.bias: 256 parameters
layers.3.fc1.weight: 32,768 parameters
layers.3.fc1.bias: 128 parameters
layers.4.fc1.weight: 8,192 parameters
layers.4.fc1.bias: 64 parameters
policy_head.weight: 768 parameters
policy_head.bias: 12 parameters
value_head.weight: 64 parameters
value_head.bias: 1 parameters
